In [ ]:
# To get max Delta(x,h) such that h is consistent with h_hat on T
def find_h1(h_hat, T, x):

    lag_terms = []
    lambs = np.linspace(0,40,num=100)
    present_model = None

    for j in range(len(lambs)):
        new_model = LogisticRegression(max_iter = 500)
        yi = []
        lamb = lambs[j]
        weights = []

        g0 = g1 = 0
        for i in range(len(x)):
            g0 += x[i,2]== 0
            g1 += x[i,2]==1

        for i in range(len(x)):
            Ci0 = (x[i,2] == 1) * 0 + (x[i,2] == 0) * (-1/g0)
            Ci1 = (x[i,2] == 1) * (-1/g1) + (x[i,2] == 0) * 0

            if i in T:
                Ci0 += -lamb * (h_hat.predict(x[i].reshape(1,-1))[0] == 0)
                Ci1 += -lamb * (h_hat.predict(x[i].reshape(1,-1))[0] == 1)

            weights.append(np.abs(Ci0 - Ci1))
            yi.append(Ci0 >= Ci1)

        new_model.fit(x, yi,sample_weight=weights)
        present_model = new_model

        lag_term = 0
        for s in T:
            lag_term += new_model.predict(x[s].reshape(1,-1)) == h_hat.predict(x[s].reshape(1,-1))
        lag_terms.append(lag_term)
        if(lag_term == len(T)):
            break

    return present_model

In [ ]:
# To get min Delta(x,h) such that h is consistent with h_hat on T
def find_h2(h_hat, T, x):

    lag_terms = []
    lambs = np.linspace(0,40,num=100)
    present_model = None

    for j in range(len(lambs)):
        new_model = LogisticRegression(max_iter=500)
        yi = []
        lamb = lambs[j]
        weights = []
        g0 = g1 = 0
        for i in range(len(x)):
            g0 += x[i,2]== 0
            g1 += x[i,2]==1
        for i in range(len(x)):
            Ci0 = (x[i,2] == 1) * 0 + (x[i,2] == 0) * (1/g0)
            Ci1 = (x[i,2] == 1) * (1/g1) + (x[i,2] == 0) * 0

            if i in T:
                Ci0 += -lamb * (h_hat.predict(x[i].reshape(1,-1))[0] == 0)
                Ci1 += -lamb * (h_hat.predict(x[i].reshape(1,-1))[0] == 1)

            weights.append(np.abs(Ci0 - Ci1))
            yi.append(Ci0 >= Ci1)

        new_model.fit(x, yi,sample_weight=weights)
        present_model = new_model
        lag_term = 0
        for s in T:
            lag_term += new_model.predict(x[s].reshape(1,-1)) == h_hat.predict(x[s].reshape(1,-1))
        lag_terms.append(lag_term)

        if(lag_term == len(T)):
            break

    return present_model

In [ ]:
def online_learning_oracle(X,predictions,S):
    model = LogisticRegression()
    y = []
    for i in S:
        y.append(predictions[i])
    y = np.array(y)
    if(len(S) > 0):
        model.fit(X[S], y.ravel())
    else:
        y = np.array([0,1]).reshape(-1,1)
        model.fit(X[[0,1]], y.ravel())
    return model

def mu(model, X):
    num_0 = 0
    num_1 = 0
    sum_0 = 0
    sum_1 = 0

    for i in range(len(X)):
        if(X[i,2]):
            num_1 += 1
            sum_1 += model.predict(X[i].reshape(1,-1))[0] == 1

        else:
            num_0 += 1
            sum_0 += model.predict(X[i].reshape(1,-1))[0] == 1

    if(num_0 > 0 and num_1 > 0): return sum_1/num_1 - sum_0/num_0
    elif(num_0 == 0): return sum_1/num_1
    return sum_0/num_0

In [ ]:
# Main Active Fairness Auditing Algorithm
def active_auditing(X,M,model,epsilon,budget):
    
    S = set()
    predictions = dict()
    budget_used = 0

    while True:
        w = [1/len(X) for _ in range(len(X))]
        fail_prob = 0.1
        tau_x = [np.random.exponential(1/(2*np.log(X.shape[1]) + np.log(M) - np.log(fail_prob))) for _ in range(len(X))]
        h_hat = online_learning_oracle(X,predictions,list(S))  
        T = set()

        while True:
            h1 = find_h1(h_hat, T, X)
            h2 = find_h2(h_hat, T, X)

            mu_diff = mu(h1,X) - mu(h2,X)
  
            if mu_diff <= 2 * epsilon:
                break  

            delta = {i for i in range(len(X)) if h1.predict(X[i].reshape(1,-1)) != h_hat.predict(X[i].reshape(1,-1)) or h2.predict(X[i].reshape(1,-1)) != h_hat.predict(X[i].reshape(1,-1))}

            while sum(w[i] for i in delta) <= 1 + 1e-3:
                for i in delta:
                    w[i] *= 2  
                T = set([i for i in range(len(x)) if w[i] >= tau_x[i]])

        diff = 0
        for el in T:
            diff += h_hat.predict(X[el].reshape(1,-1)) != model.predict(X[el].reshape(1,-1))

        
        for i in T:
            if(budget_used >= budget): break
            predictions[i] = model.predict(X[i].reshape(1,-1))[0]
            S.add(i)
            budget_used += 1

        if(budget_used >= budget):
            return S

In [ ]:
# Run this multiple times to get the mean value of the deviations
budgets = 50
M = 100
epsilon = 0.25
u_D = np.abs(mu(model, x))

S = list(active_auditing(x, M, model, epsilon, budget))
u1 = np.abs(mu(find_h1(model, S, x), x))
u2 = np.abs(mu(find_h2(model, S, x), x))
u = max(u1, u2)
u_S = np.abs(mu(model, x[S]))
print(max(u - u_D, 0))